In [2]:
import tensorflow_datasets as tfds

import tensorflow as tf
tfds.disable_progress_bar()

In [3]:
mirrored_strategy = tf.distribute.MirroredStrategy()

def get_data():
  datasets, ds_info = tfds.load(name="mnist", with_info=True, as_supervised=True)
  mnist_train, mnist_test = datasets["train"], datasets["test"]

  BUFFER_SIZE = 10000

  BATCH_SIZE_PER_REPLICA = 64
  BATCH_SIZE = BATCH_SIZE_PER_REPLICA * mirrored_strategy.num_replicas_in_sync

  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255

    return image, label
  
  train_dataset = mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
  eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

  return train_dataset, eval_dataset

def get_model():
  with mirrored_strategy.scope():
    model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation="relu", input_shape=(28,28,1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation="relu"),
      tf.keras.layers.Dense(10)                             
    ])

    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=["accuracy"])
    return model

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [4]:
model = get_model()
train_dataset, eval_dataset = get_data()
model.fit(train_dataset, epochs=2)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.



Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.0. Subsequent calls will reuse this data.
Epoch 1/2
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


...

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


938/938 [==============================] - 7s 7ms/step - accuracy: 0.9452 - loss: 0.1871
Epoch 2/2
938/938 [==============================] - 3s 3ms/step - accuracy: 0.9816 - loss: 0.0605


In [5]:
keras_model_path = "tmp/keras_save"
model.save(keras_model_path)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: tmp/keras_save/assets


INFO:tensorflow:Assets written to: tmp/keras_save/assets


In [6]:
restored_keras_model = tf.keras.models.load_model(keras_model_path)
restored_keras_model.fit(train_dataset, epochs=2)

Epoch 1/2
938/938 [==============================] - 2s 3ms/step - loss: 0.0443 - accuracy: 0.9861
Epoch 2/2
938/938 [==============================] - 2s 2ms/step - loss: 0.0322 - accuracy: 0.9902


In [8]:
another_strategy = tf.distribute.OneDeviceStrategy("/cpu:0")
with another_strategy.scope():
  restored_keras_model_ds = tf.keras.models.load_model(keras_model_path)
  restored_keras_model_ds.fit(train_dataset, epochs=2)

Epoch 1/2
938/938 [==============================] - 19s 20ms/step - loss: 0.0451 - accuracy: 0.9862
Epoch 2/2
938/938 [==============================] - 19s 20ms/step - loss: 0.0312 - accuracy: 0.9906


In [9]:
model = get_model()
saved_model_path = "tmp/tf_save"
tf.saved_model.save(model, saved_model_path)

INFO:tensorflow:Assets written to: tmp/tf_save/assets


INFO:tensorflow:Assets written to: tmp/tf_save/assets


In [10]:
DEFAULT_FUNCTION_KEY = "serving_default"
loaded = tf.saved_model.load(saved_model_path)
inference_func = loaded.signatures[DEFAULT_FUNCTION_KEY]

In [11]:
predict_dataset = eval_dataset.map(lambda image, label: image)
for batch in predict_dataset.take(1):
  print(inference_func(batch))

{'dense_3': <tf.Tensor: shape=(64, 10), dtype=float32, numpy=
array([[-1.12064295e-01, -5.19634411e-02,  1.19339138e-01,
         4.59170267e-02, -1.59013048e-02,  5.89837208e-02,
         7.68163279e-02,  9.72576737e-02, -1.14359848e-01,
         9.11261514e-03],
       [ 2.67654657e-04,  2.79976651e-02,  1.48698777e-01,
         1.88628256e-01,  5.22958562e-02,  4.62939143e-02,
         8.91167521e-02,  1.14625774e-01,  4.98715937e-02,
        -7.29172453e-02],
       [-3.49866785e-02, -6.45557493e-02,  1.78777337e-01,
         1.15453571e-01, -9.04779658e-02, -1.43328123e-03,
         1.23820364e-01,  3.85451652e-02, -7.67812729e-02,
         1.26923695e-02],
       [ 7.97091424e-03,  1.19586207e-01,  3.86014059e-02,
         4.58077714e-03, -5.82271591e-02,  6.32626787e-02,
         9.66437533e-03,  1.05679572e-01, -3.72924805e-02,
        -9.15333033e-02],
       [ 4.91752997e-02,  2.54643932e-02,  1.32490918e-01,
         1.83511078e-02,  1.64997876e-02,  1.05574325e-01,
        

In [12]:
another_strategy = tf.distribute.MirroredStrategy()
with another_strategy.scope():
  loaded = tf.saved_model.load(saved_model_path)
  inference_func = loaded.signatures[DEFAULT_FUNCTION_KEY]

  dist_predict_dataset = another_strategy.experimental_distribute_dataset(predict_dataset)

  for batch in dist_predict_dataset:
    another_strategy.run(inference_func, args=(batch,))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [13]:
import tensorflow_hub as hub

def build_model(loaded):
  x = tf.keras.layers.Input(shape=(28,28,1), name="input_x")

  keras_layer = hub.KerasLayer(loaded, trainable=True)(x)
  model = tf.keras.Model(x, keras_layer)
  return model

another_strategy = tf.distribute.MirroredStrategy()
with another_strategy.scope():
  loaded = tf.saved_model.load(saved_model_path)
  model = build_model(loaded)

  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
  model.fit(train_dataset, epochs=2)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


Epoch 1/2
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


938/938 [==============================] - 3s 3ms/step - accuracy: 0.9422 - loss: 0.2000
Epoch 2/2
938/938 [==============================] - 3s 3ms/step - accuracy: 0.9817 - loss: 0.0616


In [14]:
model = get_model()

model.save(keras_model_path)

another_strategy = tf.distribute.MirroredStrategy()

with another_strategy.scope():
  loaded = tf.saved_model.load(keras_model_path)

INFO:tensorflow:Assets written to: tmp/keras_save/assets


INFO:tensorflow:Assets written to: tmp/keras_save/assets


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [15]:
class SubclassedModel(tf.keras.Model):
  
  output_name = "output_layer"

  def __init__(self):
    super(SubclassedModel, self).__init__()
    self._dense_layer = tf.keras.layers.Dense(5, dtype=tf.dtypes.float32, name=self.output_name)
  
  def call(self, inputs):
    return self._dense_layer(inputs)

my_model = SubclassedModel()

tf.saved_model.save(my_model, saved_model_path)

INFO:tensorflow:Assets written to: tmp/tf_save/assets


INFO:tensorflow:Assets written to: tmp/tf_save/assets
